In [2]:
# Import the required package to run the deforestation risk
import os
from shutil import copy2
import urllib.request
from zipfile import ZipFile

import forestatrisk as far

In [2]:
# Make output directory
far.make_dir("output")

## 1.3 Sampling the Observations

In [3]:
# Sample points
dataset = far.sample(nsamp=10000, adapt=True, seed=1234, csize=10,
                     var_dir="data",
                     input_forest_raster="fcc23.tif",
                     output_file="output/sample.txt",
                     blk_rows=0)

Sample 2x 10000 pixels (deforested vs. forest)
Divide region in 385 blocks
Compute number of deforested and forest pixels per block
100%
Draw blocks at random
Draw pixels at random in blocks
100%
Compute center of pixel coordinates
Compute number of 10 x 10 km spatial cells
... 243 cells (9 x 27)
Identify cell number from XY coordinates
Make virtual raster with variables as raster bands
Extract raster values for selected pixels
100%
Export results to file output/sample.txt


In [4]:
# Remove NA from data-set (otherwise scale() and
# model_binomial_iCAR do not work)
dataset = dataset.dropna(axis=0)
# Set number of trials to one for far.model_binomial_iCAR()
dataset["trial"] = 1
# Print the first five rows
print(dataset.head(5))

   altitude  dist_defor   dist_edge   dist_river    dist_town  fcc23   pa  \
0     214.0  228.473190   30.000000   182.482880   295.465729    0.0  0.0   
1     164.0   30.000000   30.000000    30.000000   152.970581    0.0  0.0   
2     361.0  189.736664  189.736664   931.933472   720.624756    0.0  0.0   
3     550.0  641.326721  391.152130  1033.150513  1129.291870    0.0  0.0   
4     254.0  218.403290   30.000000   502.891632   618.465820    0.0  0.0   

       slope             X              Y  cell  trial  
0   9.463165  1.358587e+07  107957.919817   9.0      1  
1  10.137362  1.358677e+07  108557.919817   9.0      1  
2  19.261900  1.358743e+07  107777.919817   9.0      1  
3  11.140893  1.358479e+07  107807.919817   9.0      1  
4  14.135473  1.358557e+07  108077.919817   9.0      1  


## 2.1 Model Preparation

In [5]:
# Neighborhood for spatial-autocorrelation
nneigh, adj = far.cellneigh(raster="data/fcc23.tif", csize=10, rank=1)

# List of variables
#variables = ["(altitude)", "(slope)",
             #"(dist_defor)", "(dist_edge)", "(dist_road)",
             #"(dist_town)", "(dist_river)"]

#List of variables
variables = ["scale(altitude)", "scale(slope)",
             "scale(dist_defor)", "scale(dist_edge)",
             "scale(dist_town)", "scale(dist_river)", "C(pa)"]

# Formula
right_part = " + ".join(variables) + " + cell"
left_part = "I(1-fcc23) + trial ~ "
formula = left_part + right_part

# Starting values
beta_start = -99  # Simple GLM estimates

# Priors
priorVrho = -1  # -1="1/Gamma"

Compute number of 10 x 10 km spatial cells
... 243 cells (9 x 27)
Identify adjacent cells and compute number of neighbors


## 2.2 iCAR model

In [6]:
# Run the model
mod_binomial_iCAR = far.model_binomial_iCAR(
    # Observations
    suitability_formula=formula, data=dataset,
    # Spatial structure
    n_neighbors=nneigh, neighbors=adj,
    # Priors
    priorVrho=priorVrho,
    # Chains
    burnin=1000, mcmc=1000, thin=1,
    # Starting values
    beta_start=beta_start)

C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  

Using estimates from classic logistic regression as starting values for betas


## 2.3 Model Summary

In [7]:
# Predictions
pred_icar = mod_binomial_iCAR.theta_pred

# Summary
print(mod_binomial_iCAR)
# Write summary in file
with open("output/summary_icar.txt", "w") as f:
    f.write(str(mod_binomial_iCAR))

Binomial logistic regression with iCAR process
  Model: I(1 - fcc23) + trial ~ 1 + C(pa) + scale(altitude) + scale(slope) + scale(dist_defor) + scale(dist_edge) + scale(dist_town) + scale(dist_river) + cell
  Posteriors:
                        Mean        Std     CI_low    CI_high
        Intercept      -2.56      0.145      -2.85      -2.27
     C(pa)[T.1.0]     -0.446      0.139     -0.722     -0.182
  scale(altitude)     -0.276     0.0864     -0.454     -0.115
     scale(slope)      -0.34     0.0277     -0.391     -0.286
scale(dist_defor)     -0.945      0.115      -1.16     -0.725
 scale(dist_edge)      -3.26      0.205      -3.64      -2.88
 scale(dist_town)      -2.29      0.136      -2.58      -2.04
scale(dist_river)      0.052     0.0502    -0.0499      0.146
             Vrho       6.66       1.21       4.67       9.57
         Deviance   1.16e+04       26.3   1.15e+04   1.16e+04



## 3.1 Interpolate Spatial Random Effects

In [8]:
# Spatial random effects
rho = mod_binomial_iCAR.rho

# Interpolate
far.interpolate_rho(rho=rho, input_raster="data/fcc23.tif",
                    output_file="output/rho.tif",
                    csize_orig=10, csize_new=1)

Write spatial random effect data to disk
Compute statistics
Build overview
Resampling spatial random effects to file output/rho.tif


## 3.2 Predict Deforestation Probability

In [9]:
# Update dist_edge and dist_defor at t3
os.rename("data/dist_edge.tif", "data/dist_edge.tif.bak")
os.rename("data/dist_defor.tif", "data/dist_defor.tif.bak")
os.rename("data/dist_town.tif", "data/dist_town.tif.bak")
copy2("data/forecast/dist_edge_forecast.tif", "data/dist_edge.tif")
copy2("data/forecast/dist_defor_forecast.tif", "data/dist_defor.tif")
copy2("data/forecast/dist_town_forecast.tif", "data/dist_town.tif")

# Compute predictions
far.predict_raster_binomial_iCAR(
    mod_binomial_iCAR, var_dir="data",
    input_cell_raster="output/rho.tif",
    input_forest_raster="data/forest/forest_t3.tif",
    output_file="output/prob.tif",
    blk_rows=10  # Reduced number of lines to avoid memory problems
)

# Reinitialize data
os.remove("data/dist_edge.tif")
os.remove("data/dist_defor.tif")
os.remove("data/dist_town.tif")
os.rename("data/dist_edge.tif.bak", "data/dist_edge.tif")
os.rename("data/dist_defor.tif.bak", "data/dist_defor.tif")
os.rename("data/dist_town.tif.bak", "data/dist_town.tif")

Make virtual raster with variables as raster bands
Divide region in 271 blocks
Create a raster file on disk for projections
Predict deforestation probability by block
1%

C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  

2%

C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  

3%

...

98%

C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  

100%


C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  

Compute statistics


## 4 Project Future Forest Cover Change

In [10]:
# Forest cover
fc = list()
dates = ["t2", "t3"]
ndates = len(dates)
for i in range(ndates):
    rast = "data/forest/forest_" + dates[i] + ".tif"
    val = far.countpix(input_raster=rast, value=1)
    fc.append(val["area"])  # area in ha
# Save results to disk
f = open("output/forest_cover.txt", "w")
for i in fc:
    f.write(str(i) + "\n")
f.close()
# Annual deforestation
T = 10.0
annual_defor = (fc[0] - fc[1]) / T
print("Mean annual deforested area during the period 2010-2020: {} ha/yr".format(annual_defor))

Divide region in 385 blocks
Compute the number of pixels with value=1
100%
Compute the corresponding area in ha
Divide region in 385 blocks
Compute the number of pixels with value=1
100%
Compute the corresponding area in ha
Mean annual deforested area during the period 2010-2020: 6264.180000000005 ha/yr


In [11]:
# Projected deforestation (ha) during 2020-2050
defor = annual_defor * 30

# Compute future forest cover in 2050
stats = far.deforest(
    input_raster="output/prob.tif",
    hectares=defor,
    output_file="output/fcc_2050.tif",
    blk_rows=128)

Identify threshold
Minimize error on deforested hectares
Create a raster file on disk for forest-cover change
Divide region in 22 blocks
Write raster of future forest-cover change
100%
Compute statistics


## 5.1 Historical Forest Cover Change

In [12]:
# Plot forest
fig_fcc123 = far.plot.fcc123(
    input_fcc_raster="data/forest/fcc123.tif",
    maxpixels=1e4,
    output_file="output/fcc123.png",
    borders="data/ctry_PROJ.shp",
    linewidth=0.2,
    figsize=(5, 4), dpi=800)

Build overview


AttributeError: 'NoneType' object has no attribute 'GetLayer'

## 5.2 Spatial Random Effects

In [ ]:
# Original spatial random effects
fig_rho_orig = far.plot.rho("output/rho_orig.tif",
                            borders="data/ctry_PROJ.shp",
                            linewidth=0.5,
                            output_file="output/rho_orig.png",
                            figsize=(9,5), dpi=80)

# Interpolated spatial random effects
fig_rho = far.plot.rho("output/rho.tif",
                       borders="data/ctry_PROJ.shp",
                       linewidth=0.5,
                       output_file="output/rho.png",
                       figsize=(9,5), dpi=80)

## 5.3 Spatial Probability of Deforestation

In [ ]:
# Spatial probability of deforestation
fig_prob = far.plot.prob("output/prob.tif",
                         maxpixels=1e4,
                         borders="data/ctry_PROJ.shp",
                         linewidth=0.2,
                         legend=True,
                         output_file="output/prob.png",
                         figsize=(5, 4), dpi=800)

## 5.4 Future Forest Cover

In [ ]:
# Projected forest cover change (2020-2050)
fcc_2050 = far.plot.fcc("output/fcc_2050.tif",
                        maxpixels=1e4,
                        borders="data/ctry_PROJ.shp",
                        linewidth=0.2,
                        output_file="output/fcc_2050.png",
                        figsize=(5, 4), dpi=800)